## Analysis_BAT2

In [14]:
    
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import xml.etree.ElementTree as et 

print('pd.__version__', pd.__version__)
print('np.__version__', np.__version__)
print('sns.__version__', sns.__version__)

print(pd.get_option("display.max_rows"), 
    pd.get_option("display.max_columns"), 
    pd.get_option("display.max_colwidth"))
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_colwidth", 200)

cwd = '/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_JG_BATTELLE_PART2/VAST_images'
cwd = '/Volumes/GoogleDrive/Shared drives/BATTELLE_2021_PART2/ZC_2021_JG_BATTELLE_PART2/VAST_images'

pd.__version__ 1.2.4
np.__version__ 1.20.3
sns.__version__ 0.11.2
200 200 200


### Zetemplate done - plate xml is there

In [15]:
zetemplate_done, zetemplate_pending = [], []
for platefolder in os.listdir(cwd): 
    if os.path.isfile(os.path.join(cwd, platefolder, platefolder+'.xml')):
        zetemplate_done.append(platefolder)
    else:
        zetemplate_pending.append(platefolder)        
print('total platefolders: ', len(zetemplate_done)+len(zetemplate_pending), ', zetemplate_done: ', len(zetemplate_done))
toDo_list = pd.DataFrame(data=zetemplate_done+zetemplate_pending, columns=['plate'])

toDo_list.loc[toDo_list['plate'].isin(zetemplate_done), 'zetemplate_done'] = True
toDo_list.head()
# total platefolders:  210 , zetemplate_done:  201
# total platefolders:  257 , zetemplate_done:  247
# total platefolders:  267 , zetemplate_done:  254

total platefolders:  269 , zetemplate_done:  256


,plate,zetemplate_done
0,20220423_1747_DM_R2,True
1,20220402_1271_DM_R1,True
2,20220402_1584_DM_R1,True
3,20220405_1895_DM_R3,True
4,20220423_1477_DM_R2,True


### read status csv - exclude excluded plates

In [16]:
# BAT2_process_status_20201209_comments.csv - 15 plates excluded?
# load latest status csv 
last_df = pd.read_csv(os.path.join(os.path.split(cwd)[0], 'BAT2_progress.csv'), sep=',')
last_df = last_df[['EXCLUDED', 'plate_name']].copy()
print(last_df['EXCLUDED'].unique())

# excluded plates
excluded_plates = last_df[last_df['EXCLUDED'].notnull()]['plate_name']
print(len(excluded_plates), 'plates excluded')

# plates to analyse
plates2analyse = last_df[~last_df['plate_name'].isin(excluded_plates)]['plate_name']

# ---- > analyse only DM not DS - already done
plates2analyse = [item for item in plates2analyse if '_DS_' in item]

# drop compound 1547
#plates2analyse = [item for item in plates2analyse if '1547' not in item]

print(len(plates2analyse), 'plates to analyse', 'should have:', 41*3)

[nan 'EXCLUDED']
23 plates excluded
123 plates to analyse should have: 123


# phenotype children
## every phenotype has a status "finished" if finished. 

***Boolean phenotypes***

**value**: True or False

- bodycurvature
- snoutjawdefects
- yolkedema
- necrosis
- tailbending
- notochorddefects
- craniofacialedema
- finabsence
- scoliosis

***Measured phenotypes***

- **fishoutline** 
    - **dorsal_image_area** 
    - **lateral_image_area**
    - **unit** "micron_square"
    
Fishoutline is computed on the dorsal and lateral image.
Fishoutline is prerequisite for all subsequent measured phenotypes.

- **eyes** 
    - **dorsal_image_eye_down_length**
    - **dorsal_image_eye_up_length**
    - **unit** "micron"    
    - eyes_down_params (eye pairs generated by algo, to toggle from)
    - eyes_up_params (eye pairs generated by algo, to toggle from)
    
Eyes are quantified in dorsal images here, but may eventually be quantified in lateral images as well.
For eyes the diameter length is measured from the selected ellipse. 
The ellipse is the one saved in the .roi file. 
It's not neccesarily present in eyes_down_params, eyes_up_params, as it may be user modified. 

- **pigmentation** 
    - **dorsal_image_area_fish_wo_eyes** area of the dorsal fish without eyes
    - **dorsal_image_area_pigmentation** area of pigments 
    - **dorsal_image_area_ratio** ratio of pigments/ total area (fish without eyes)
    - **unit** "micron_square"  
    - **threshold** "45"

for pigmentation eyes are generally excluded. Necrosis can lead to falsely detected pigmentation. 
Pigmentation area is derived here from applying a fixed threshold. For normalization it's to be seen if and which
normalization will make sense (dorsal fish without eyes, bodylenght...)

- **heart** 
    - **lateral_image_area**
    - **unit** "micron_square"    

- **bodylength** 
    - **lateral_image_length**  >
    - **unit** "micron"    

# Conventions: 

- There can be either dead24 OR dead120 selected in ZeTemplate. That is to signify the timepoint at which death was assessed and seen. Here mutually exclusive for 24 and 120. 
But for Battelle the fields are to signify simply whether an embryo is dead or alive at the timepoint. And here every embryo dead at 24 is also dead at 120 (we don't check for spontaneous reanimation...)


# Their naming convention

- **Axis: curvature of body axis** --> body_axis_defect
- **Yolk sac: Edema** --> yolk_sac_edema


- **Craniofacial: eye** defects --> unspecific, enlarged or decreased
- **Craniofacial: snout jaw defects** --> snoutjawdefects
- **Craniofacial: edema** --> craniofacial_edema


- **Percardial tissue: Edema** --> pericardial_area enlarged


- **Trunk: short** --> body_length decreased
- **Trunk: long** --> body_length increased


- **Pigment: abnormal** --> pigmentation increased or decreased
- **Pigment: decreased** --> pigmentation decreased
- **Pigment: absent** coloration --> pigmentation smaller than?? not clear --> nothing

# ---- FUNCTIONS -----

In [17]:
well_attributes = ["chorion", "exposure", "compound", "dose", "unit", "dead24", "dead120", "unhatched120"
                   , "creation_date", "well_folder", "lateral_image", "dorsal_image", "show2user", "excluded"
                   , "correction_done"]

booleans = ['bodycurvature', 'snoutjawdefects', 'yolkedema', 'necrosis', 'tailbending', 'notochorddefects'
            , 'craniofacialedema', 'finabsence', 'scoliosis']

measured = ['fishoutline', 'eyes', 'pigmentation', 'heart', 'bodylength']

params_measured = {'fishoutline':['dorsal_image_area', 'lateral_image_area', 'unit']
                   , 'eyes':['dorsal_image_eye_down_length', 'dorsal_image_eye_up_length']
                   , 'pigmentation':['dorsal_image_area_fish_wo_eyes', 'dorsal_image_area_pigmentation', 'dorsal_image_area_ratio', 'unit', 'threshold']
                   , 'heart':['lateral_image_area', 'unit']
                   , 'bodylength':['lateral_image_length', 'unit']}


def parse_xml_full(xml_path, well_attributes, booleans, measured, params_measured):
    '''get all BAT2 phenotypes /values out - not checking for status here'''
    rows = []
    xtree = et.parse(xml_path)
    xroot = xtree.getroot() 
    plate_name = xroot.attrib.get("plate_name")
    wells = []
    
    measured_cols = [item+'_'+att for item in list(params_measured.keys()) for att in list(params_measured[item])]
    
    cols = ['plate_name', 'name'] + well_attributes + booleans + measured_cols
    
    
    for node in xroot: 
        well_name = node.attrib.get("name")
        wells.append(well_name)
    
    for well in wells: 
        d = {}
        thiswell = xroot.find(".//*[@name='"+well+"']")
        d['plate_name'] = plate_name
        d['name']=well
        
        # general attributes 
        for item in well_attributes: 
            d[item] = thiswell.attrib.get(item)
        
        # boolean pts
        for bo in booleans: 
            element = thiswell.find(bo)
            if element != None:
                d[bo] = element.get('value')
            else: 
                d[bo] = 'None'
           
        # measured: 
        for me in measured: 
            element = thiswell.find(me)
            for att in params_measured[me]:        
                if element != None:
                    d[me+'_'+att] = element.get(att)
                else: 
                    d[me+'_'+att] = None
        
        # append the dict            
        rows.append(d)
    
    out_df = pd.DataFrame(rows, columns = cols)
    
    return out_df

    ## preprocess
def preprocess(df1):
    df = df1.copy()
    # set well_name
    df.rename(columns={'name':'well_name'}, inplace=True)

    # make condtion column
    df['condition'] = df['compound']+'_'+df['dose'].astype(str)+'_'+df['unit']

    # pd to num
    for col in df.columns: 
        df[col] = pd.to_numeric(df[col], errors='ignore')

    # get eye diameter mean
    df['avg_eye_diameter'] = (df['eyes_dorsal_image_eye_down_length']+df['eyes_dorsal_image_eye_up_length'])/2
    
    # dead120 True if dead24 True
    df.loc[df['dead24']==1, 'dead120'] = 1
    
    # set excluded wells value to nan
    columns = list(df.columns)
    col2keep = well_attributes + ['plate_name', 'well_name', 'condition']
    col2nan = [c for c in columns if c not in col2keep]
    df.loc[df['excluded']=='True', col2nan] = np.nan
    
    return df

def phenotype_from_continous(df1, neg_CTRL_identifier, phenotype_col, plot_stuff): 
    '''check a measured phenotype against negCtrl (searchstring) and provide battelle format output
    return a figure if desired'''
    df = df1.copy()
    neg_CTRL_name = [name for name in df.condition if neg_CTRL_identifier in name][0]
    grouped = df.groupby(['condition']).agg(['mean', 'std'])
    
    ul = grouped[(phenotype_col, 'mean')][neg_CTRL_name] + 2*grouped[(phenotype_col, 'std')][neg_CTRL_name]
    ll = grouped[(phenotype_col, 'mean')][neg_CTRL_name] - 2*grouped[(phenotype_col, 'std')][neg_CTRL_name]
    
    #print('ll: ', ll, 'ul: ', ul)
    
    def enlarged_or_decreased(x, ul, ll):
        if x > ul or x<ll:
            return True
        elif x < ul and x>ll:
            return False
        else: 
            return x
    
    def enlarged(x, ul, ll): 
        if x > ul: 
            return True
        elif x < ul: 
            return False
        else: return x
    
    def decreased(x, ul, ll): 
        if x < ll: 
            return True
        elif x > ll: 
            return False
        else: return x
    
    if phenotype_col == 'avg_eye_diameter':
        #Craniofacial: eye defects --> unspecific, enlarged or decreased
        df['Craniofacial: eye defects'] = df[phenotype_col].apply(lambda x: enlarged_or_decreased(x, ul, ll))        
    #if phenotype_col == 'pigmentation_dorsal_image_area_pigmentation':
    if 'pigmentation' in phenotype_col: 
        #'pigmentation_dorsal_image_area_pigmentation',
        #'pigmentation_dorsal_image_area_ratio'
        #Pigment: abnormal --> pigmentation increased or decreased
        df['Pigment: abnormal'] = df[phenotype_col].apply(lambda x: enlarged_or_decreased(x, ul, ll))
        df['Pigment: decreased'] = df[phenotype_col].apply(lambda x: decreased(x, ul, ll))
        df['Pigment: absent coloration'] = np.nan
    
    if phenotype_col == 'heart_lateral_image_area':
        #Percardial tissue: Edema --> pericardial_area enlarged
        df['Percardial tissue: Edema'] = df[phenotype_col].apply(lambda x: enlarged(x, ul, ll))

    if phenotype_col == 'bodylength_lateral_image_length':
        #Trunk: short --> body_length decreased
        #Trunk: long --> body_length increased
        df['Trunk: short'] = df[phenotype_col].apply(lambda x: decreased(x, ul, ll))
        df['Trunk: long'] = df[phenotype_col].apply(lambda x: enlarged(x, ul, ll))
        
    fig = None
    if plot_stuff: 
        fig = plt.figure(figsize=(4, 4))
        ax = sns.boxplot(x = 'condition', y=phenotype_col, data=df[~df.condition.str.contains('DCA')], color='limegreen')
        ax = sns.swarmplot(x = 'condition', y=phenotype_col, data=df[~df.condition.str.contains('DCA')], color='black')
        ax.set_xlabel('')
        labels = ax.get_xticklabels()
        labels = [l.get_text() for l in labels]
        labels = [l.replace('_', ' ') for l in labels]
        ax.set_xticklabels(labels, rotation=90)
        plt.axhline(y=float(ul), xmin=0, xmax=1, linestyle='--', color='magenta', lw=1)
        plt.axhline(y=float(ll), xmin=0, xmax=1, linestyle='--', color='magenta', lw=1)
    return fig, df
#figure, df1 = phenotype_from_continous(df, 'DMSO', 'avg_eye_diameter', True)

# pigmentation_dorsal_image_area_pigmentation, pigmentation_dorsal_image_area_ratio
#figure, df1 = phenotype_from_continous(df, 'DMSO', 'pigmentation_dorsal_image_area_pigmentation', True) 

#figure, df1 = phenotype_from_continous(df, 'DMSO', 'heart_lateral_image_area', True) 

#figure, df1 = phenotype_from_continous(df, 'DMSO', 'bodylength_lateral_image_length', True) 


def phenotype_from_continous_different_neg_ctrl(df_comp, df_neg_ctrl, neg_CTRL_identifier, phenotype_col): 
    '''check a measured phenotype against a negCtrl (searchstring) ON A DIFFERENT PLATE and provide battelle format output'''
    df = df_neg_ctrl.copy()
    neg_CTRL_name = [name for name in df.condition if neg_CTRL_identifier in name][0]
    grouped = df.groupby(['condition']).agg(['mean', 'std'])
    
    ul = grouped[(phenotype_col, 'mean')][neg_CTRL_name] + 2*grouped[(phenotype_col, 'std')][neg_CTRL_name]
    ll = grouped[(phenotype_col, 'mean')][neg_CTRL_name] - 2*grouped[(phenotype_col, 'std')][neg_CTRL_name]
    
    #print('ll: ', ll, 'ul: ', ul)
    
    def enlarged_or_decreased(x, ul, ll):
        if x > ul or x<ll:
            return True
        elif x < ul and x>ll:
            return False
        else: 
            return x
    
    def enlarged(x, ul, ll): 
        if x > ul: 
            return True
        elif x < ul: 
            return False
        else: return x
    
    def decreased(x, ul, ll): 
        if x < ll: 
            return True
        elif x > ll: 
            return False
        else: return x
    
    if phenotype_col == 'avg_eye_diameter':
        #Craniofacial: eye defects --> unspecific, enlarged or decreased
        df_comp['Craniofacial: eye defects'] = df_comp[phenotype_col].apply(lambda x: enlarged_or_decreased(x, ul, ll))        
    #if phenotype_col == 'pigmentation_dorsal_image_area_pigmentation':
    if 'pigmentation' in phenotype_col: 
        #'pigmentation_dorsal_image_area_pigmentation',
        #'pigmentation_dorsal_image_area_ratio'
        #Pigment: abnormal --> pigmentation increased or decreased
        df_comp['Pigment: abnormal'] = df_comp[phenotype_col].apply(lambda x: enlarged_or_decreased(x, ul, ll))
        df_comp['Pigment: decreased'] = df_comp[phenotype_col].apply(lambda x: decreased(x, ul, ll))
        df_comp['Pigment: absent coloration'] = np.nan
    
    if phenotype_col == 'heart_lateral_image_area':
        #Percardial tissue: Edema --> pericardial_area enlarged
        df_comp['Percardial tissue: Edema'] = df_comp[phenotype_col].apply(lambda x: enlarged(x, ul, ll))

    if phenotype_col == 'bodylength_lateral_image_length':
        #Trunk: short --> body_length decreased
        #Trunk: long --> body_length increased
        df_comp['Trunk: short'] = df_comp[phenotype_col].apply(lambda x: decreased(x, ul, ll))
        df_comp['Trunk: long'] = df_comp[phenotype_col].apply(lambda x: enlarged(x, ul, ll))
        
    return df_comp

# ---- WORKFLOW -----

## generate plate csvs

In [20]:
# if we need to run only a subset of plates: 

subset_plates = [item for item in plates2analyse if '1271' in item]
print(subset_plates)

for p, plate in enumerate(plates2analyse): 
# for p, plate in enumerate(subset_plates): 
    path = os.path.join(cwd, plate, plate+'.xml')
    print(p, 'working with: ', plate)
    df = parse_xml_full(path, well_attributes, booleans, measured, params_measured)
    df = preprocess(df)
    # measured phenotypes
    m =['avg_eye_diameter', 'bodylength_lateral_image_length'
        , 'heart_lateral_image_area', 'pigmentation_dorsal_image_area_pigmentation']
    for moo in m: 
        figure, df = phenotype_from_continous(df, 'DMSO', moo, False) 
    print(df.shape)
    outdir = os.path.split(cwd)[0]
    print('exists: ', os.path.isfile(os.path.join(outdir, 'plate_csvs', plate+'.csv')))
    df.to_csv(os.path.join(outdir, 'plate_csvs', plate+'.csv'), sep=',', index=0)

['20211221_1271_DS_R3', '20220110_1271_DS_R5', '20220115_1271_DS_R6']
0 working with:  20211127_1046_DS_R1
(96, 49)
exists:  False
1 working with:  20211127_1694_DS_R1
(96, 49)
exists:  False
2 working with:  20211127_1951_DS_R1
(96, 49)
exists:  False
3 working with:  20211127_1956_DS_R1
(96, 49)
exists:  False
4 working with:  20211129_1046_DS_R2
(96, 49)
exists:  False
5 working with:  20211129_1694_DS_R2
(96, 49)
exists:  False
6 working with:  20211129_1951_DS_R2
(96, 49)
exists:  False
7 working with:  20211129_1956_DS_R2
(96, 49)
exists:  False
8 working with:  20211204_1046_DS_R3
(96, 49)
exists:  False
9 working with:  20211204_1694_DS_R3
(96, 49)
exists:  False
10 working with:  20211204_1951_DS_R3
(96, 49)
exists:  False
11 working with:  20211204_1956_DS_R3
(96, 49)
exists:  False
12 working with:  20211221_1171_DS_R3
(96, 49)
exists:  False
13 working with:  20211221_1271_DS_R3
(96, 49)
exists:  True
14 working with:  20211227_1584_DS_R3
(96, 49)
exists:  False
15 working 

# checkpoint - for complete analysis, run further

# actually after saving to plate_csvs folder, run BAT2_Pack10compounds, then BAT2_squash_to_format